
# Introduction to PyTorch

**Author:** Your Name  
**Course:** Intro to Machine Learning / Deep Learning  
**Last updated:** 2025-10-30

Welcome! This notebook is a hands-on, beginner-friendly introduction to **PyTorch**, a popular deep learning framework with an imperative (define-by-run) style. We'll cover:

- Tensors and basic tensor operations
- Automatic differentiation with `autograd`
- Building neural networks with `torch.nn` and `torch.optim`
- Datasets, transforms, and `DataLoader`
- Training and evaluating a simple CNN on **MNIST**

> **Prereqs:** Basic Python & NumPy, and a high-level understanding of linear algebra and neural nets.



## Learning Objectives

By the end of this notebook, you should be able to:

1. Create and manipulate tensors, move them across CPU/GPU, and understand broadcasting.
2. Use PyTorch's **autograd** to compute gradients and perform a simple gradient descent step.
3. Define models with `nn.Module` and configure an optimizer and loss function.
4. Use `torchvision.datasets` and `DataLoader` to feed data into your model.
5. Train, evaluate, save, and load a PyTorch model on the MNIST dataset.



## 0. Environment Setup

If you don't already have PyTorch and TorchVision installed, uncomment and run the following cell.  
Pick the correct command for your platform from https://pytorch.org/get-started/locally/ if needed.


In [ ]:

# !pip install --upgrade pip
# CPU-only (works on most machines):
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# GPU (CUDA): follow instructions from https://pytorch.org/get-started/locally/



## 1. Imports and Device


In [ ]:

import os, math, time, random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

seed = 42
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device



## 2. Tensors

PyTorch's core data structure is the **Tensor**: an n-dimensional array similar to NumPy's `ndarray`, but with GPU support and automatic differentiation.


In [ ]:

# Creation
a = torch.tensor([1.0, 2.0, 3.0])              # from Python list
b = torch.zeros((2,3))                          # zeros
c = torch.ones((2,3), dtype=torch.float32)      # ones with dtype
d = torch.randn((2,3))                          # from normal distribution
e = torch.arange(0, 12).reshape(3,4)            # range + reshape

print('a:', a)
print('b.shape:', b.shape, 'dtype:', b.dtype)
print('c.device:', c.device)
print('d mean/std:', d.mean().item(), d.std().item())
print('e:\n', e)

# NumPy <-> Torch
np_arr = np.array([[1,2,3],[4,5,6]], dtype=np.float32)
t_from_np = torch.from_numpy(np_arr)    # shares memory!
np_back = t_from_np.numpy()
print('Shared memory check:', np.shares_memory(np_arr, np_back))


In [ ]:

# Indexing / slicing / broadcasting
x = torch.arange(0, 12).reshape(3,4).float()
print('x:\n', x)
print('x[0, 1:3]:', x[0, 1:3])

row = x[0]             # view (no copy)
row_clone = row.clone()
row[:] = -1            # in-place change propagates (be careful!)
print('After in-place change, x:\n', x)

# Broadcasting: elementwise add a (3x1) vector to (3x4) matrix
v = torch.tensor([[10.0],[20.0],[30.0]])  # shape (3,1)
y = x + v                                 # broadcast along columns
print('Broadcasted y shape:', y.shape)


In [ ]:

# Moving data between devices
x_cpu = torch.randn(2, 2)
x_gpu = x_cpu.to(device)
print('Device of x_gpu:', x_gpu.device)

# Best practice: create tensors directly on device when appropriate
x_on_dev = torch.randn(1024, 1024, device=device)
x_on_dev = x_on_dev @ x_on_dev.T  # matrix multiply
x_on_dev.mean().item()



### ✍️ Quick Exercise

- Create a tensor `z` of shape `(4, 5)` filled with random numbers.
- Normalize each column to have mean 0 and std 1. (Use broadcasting.)
- Verify your result with `z.mean(dim=0)` and `z.std(dim=0)`.


In [ ]:

# TODO: Your code here
# z = ...
# z_norm = ...
# print(z_norm.mean(dim=0), z_norm.std(dim=0))



## 3. Automatic Differentiation with `autograd`

`torch.autograd` tracks operations on tensors with `requires_grad=True` and can compute gradients via **backpropagation** using `.backward()`.


In [ ]:

# A simple scalar function: f(w) = (w - 3)^2
w = torch.tensor([5.0], requires_grad=True)
f = (w - 3) ** 2
f.backward()                  # df/dw = 2*(w - 3) at w=5
print('w.grad:', w.grad)      # expected 4.0


In [ ]:

# Gradient Descent on a quadratic: minimize f(w) = (w - 7)^2
w = torch.tensor([0.0], requires_grad=True)
lr = 0.1
for step in range(30):
    loss = (w - 7)**2
    loss.backward()
    with torch.no_grad():
        w -= lr * w.grad      # update
        w.grad.zero_()
    if step % 5 == 0:
        print(f'step={step:02d}  w={w.item():.4f}  loss={loss.item():.6f}')
print('Converged w ≈', w.item())



> **Note:** Autograd builds a dynamic computation graph on-the-fly. Use `with torch.no_grad():` to disable gradient tracking during evaluation or parameter updates.



## 4. `nn.Module` and `torch.optim`

Let's define a tiny **Multi-Layer Perceptron (MLP)** and a training step loop skeleton.


In [ ]:

class TinyMLP(nn.Module):
    def __init__(self, in_features=784, hidden=128, num_classes=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features, hidden),
            nn.ReLU(),
            nn.Linear(hidden, num_classes)
        )
    def forward(self, x):
        # x shape: (B, 1, 28, 28)
        x = x.view(x.size(0), -1)
        return self.net(x)

model = TinyMLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

sum(p.numel() for p in model.parameters()), model



## 5. Datasets, Transforms, and DataLoader (MNIST)

We'll use `torchvision.datasets.MNIST`. If your environment has no internet access, download the dataset beforehand or change the root path to a cached location.


In [ ]:

# Data transforms
transform = transforms.Compose([
    transforms.ToTensor(),                      # [0,1] tensor of shape (C,H,W)
    transforms.Normalize((0.1307,), (0.3081,))  # standard MNIST normalization
])

data_root = './data'  # change this if needed

# Dataset & split
train_full = datasets.MNIST(root=data_root, train=True, download=True, transform=transform)
test_ds    = datasets.MNIST(root=data_root, train=False, download=True, transform=transform)

train_size = int(0.9 * len(train_full))
val_size   = len(train_full) - train_size
train_ds, val_ds = random_split(train_full, [train_size, val_size], generator=torch.Generator().manual_seed(42))

# DataLoaders
BATCH_SIZE = 128
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

len(train_ds), len(val_ds), len(test_ds)


In [ ]:

# Peek at some samples
import matplotlib.pyplot as plt

images, labels = next(iter(train_loader))
print('Batch shape:', images.shape, 'Labels shape:', labels.shape)

grid = images[:16].squeeze(1)  # (16, 28, 28)
fig, axes = plt.subplots(4, 4, figsize=(6,6))
for ax, img, lab in zip(axes.flatten(), grid, labels[:16]):
    ax.imshow(img.numpy(), cmap='gray')
    ax.set_title(int(lab))
    ax.axis('off')
plt.tight_layout()
plt.show()



## 6. Training Utilities

We'll write small helper functions for one epoch of training and validation.


In [ ]:

def train_one_epoch(model, loader, optimizer, criterion, device='cpu'):
    model.train()
    total_loss, correct, total = 0.0, 0, 0
    for x, y in loader:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        pred = logits.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += x.size(0)

    return total_loss / total, correct / total

@torch.no_grad()
def evaluate(model, loader, criterion, device='cpu'):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0
    for x, y in loader:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        logits = model(x)
        loss = criterion(logits, y)
        total_loss += loss.item() * x.size(0)
        pred = logits.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += x.size(0)
    return total_loss / total, correct / total



## 7. Train a Simple MLP (Baseline)


In [ ]:

EPOCHS = 5
best_val_acc = 0.0
history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

for epoch in range(1, EPOCHS+1):
    t0 = time.time()
    tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    history["train_loss"].append(tr_loss)
    history["train_acc"].append(tr_acc)
    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({"model": model.state_dict()}, "mnist_mlp_best.pt")

    dt = time.time() - t0
    print(f"Epoch {epoch:02d} | {dt:.1f}s  train: loss={tr_loss:.4f}, acc={tr_acc:.4f}  "
          f"val: loss={val_loss:.4f}, acc={val_acc:.4f}")

print("Best val acc:", best_val_acc)


In [ ]:

# Plot training curves
plt.figure(figsize=(6,4))
plt.plot(history["train_loss"], label="train_loss")
plt.plot(history["val_loss"], label="val_loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend(); plt.title("Loss Curves")
plt.show()

plt.figure(figsize=(6,4))
plt.plot(history["train_acc"], label="train_acc")
plt.plot(history["val_acc"], label="val_acc")
plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.legend(); plt.title("Accuracy Curves")
plt.show()



## 8. Upgrade to a Small CNN

A small convolutional network (LeNet-style) typically performs better on MNIST.


In [ ]:

class SmallCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),  # 14x14
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),  # 7x7
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128), nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

cnn = SmallCNN().to(device)
optimizer_cnn = torch.optim.Adam(cnn.parameters(), lr=1e-3)


In [ ]:

EPOCHS_CNN = 5
best_val_acc_cnn = 0.0
for epoch in range(1, EPOCHS_CNN+1):
    tr_loss, tr_acc = train_one_epoch(cnn, train_loader, optimizer_cnn, criterion, device)
    val_loss, val_acc = evaluate(cnn, val_loader, criterion, device)
    if val_acc > best_val_acc_cnn:
        best_val_acc_cnn = val_acc
        torch.save({"model": cnn.state_dict()}, "mnist_cnn_best.pt")
    print(f"[CNN] Epoch {epoch:02d}  train: loss={tr_loss:.4f}, acc={tr_acc:.4f}  "
          f"val: loss={val_loss:.4f}, acc={val_acc:.4f}")
print("Best val acc (CNN):", best_val_acc_cnn)



## 9. Evaluate on Test Set and Inspect Predictions


In [ ]:

@torch.no_grad()
def predict(model, loader, device='cpu'):
    model.eval()
    preds, targets = [], []
    for x, y in loader:
        x = x.to(device)
        logits = model(x)
        preds.append(logits.argmax(dim=1).cpu())
        targets.append(y)
    return torch.cat(preds), torch.cat(targets)

test_loss_mlp, test_acc_mlp = evaluate(model, test_loader, criterion, device)
test_loss_cnn, test_acc_cnn = evaluate(cnn,   test_loader, criterion, device)
print(f"MLP Test: loss={test_loss_mlp:.4f}, acc={test_acc_mlp:.4f}")
print(f"CNN Test: loss={test_loss_cnn:.4f}, acc={test_acc_cnn:.4f}")

preds, targets = predict(cnn, test_loader, device)
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(targets.numpy(), preds.numpy())
print(classification_report(targets.numpy(), preds.numpy()))


In [ ]:

# Plot confusion matrix
import itertools

plt.figure(figsize=(6,6))
plt.imshow(cm, interpolation='nearest', cmap='Blues')
plt.title('Confusion Matrix (CNN)')
plt.colorbar()
tick_marks = np.arange(10)
plt.xticks(tick_marks, tick_marks)
plt.yticks(tick_marks, tick_marks)

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()



## 10. Saving and Loading Models

Use `torch.save` / `torch.load` with state dicts for portability.


In [ ]:

# Save
torch.save({'model': cnn.state_dict()}, 'mnist_cnn_checkpoint.pt')

# Load
state = torch.load('mnist_cnn_checkpoint.pt', map_location=device)
cnn_loaded = SmallCNN().to(device)
cnn_loaded.load_state_dict(state['model'])
_ = cnn_loaded.eval()
print('Model reloaded and set to eval mode.')



## 11. Tips, Tricks, and Next Steps

- **`model.train()` vs `model.eval()`**: Affects layers like `Dropout` and `BatchNorm`.
- **Mixed Precision**: `torch.cuda.amp.autocast()` can speed up training on GPUs.
- **LR Scheduling**: Explore `torch.optim.lr_scheduler` for better training dynamics.
- **Hooks**: Use forward/backward hooks for introspection.
- **Debugging**: Start with a tiny subset of data and make sure the model can overfit it.
- **Reading**: The official tutorials and docs are excellent, and the MNIST example is a good baseline for new architectures.

### 🔧 Stretch Exercises
1. Add **BatchNorm** to the CNN and compare validation accuracy.
2. Replace `ReLU` with **LeakyReLU** or **GELU** and observe changes.
3. Implement **early stopping** and/or a **ReduceLROnPlateau** scheduler.
4. Visualize learned filters in the first conv layer.



## Appendix: Troubleshooting

- **Dataset download issues**: If your environment blocks downloads, pre-download MNIST and set `root` to that folder, or copy from a machine with internet.
- **CUDA errors**: Ensure correct CUDA/PyTorch versions. Try CPU-only wheels if necessary.
- **Performance**: Increase `num_workers`, enable `pin_memory=True` for faster host->device transfers, and switch to mixed precision on GPUs.
